# Multiview ball extrinsic calibration
Pre-request, you have run step1 intrinsic calibration. The `intrinsics_calib_carl.json` is previous outcome.

In [1]:
# import the necessary packages
import sys
sys.path.append('/home/gxj/Desktop/gxj/multiview_ball_calib/2A_ball_extrinsic_calib')
import pickle
import numpy as np
import json

intrinsics_json = '/home/gxj/Desktop/gxj/calibration_playground/intrinsic_calib_frames/intrinsics_calib.json'
#intrinsics_json = '/home/gxj/Desktop/gxj/calibration_playground/calibration_20231201/intrinsic_calib_frames/intrinsics_calib.json'

## The input ball location data format
The ball position in all view is stored in a `keypoints` variable, which is (NVIEW, NSAMPLE, NKEYPOINT=1, XYP=3).

And the `P` in `XYP` is **0<=P<=1**, means the reliable of this detection. If you failed to detect a ball sometimes in a view, just set the `XY=np.nan`, and `P=0` for **IVIEW**. The `P` will be cutoff around `0.7` to auto delete reliable ball detections.

In [2]:
matfile = '/home/gxj/Desktop/gxj/calibration_playground/ball_label/ball.matpkl'
matdata = pickle.load(open(matfile, 'rb'))
assert 'keypoints' in matdata.keys()

print(matdata['keypoints'].shape) #(NVIEW, NSAMPLE, NKEYPOINT=1, xyp=3)
print(matdata['info'])            # The grid video. If you video is not grid. Just keep 'nview', 'fps'
print(matdata['views_xywh'])      # (NVIEW, 4) x,y,w,h. If you video is not grid in mycase, you just set x=0,y=0

(4, 100, 1, 3)
{'nview': 4, 'fps': 30}
[[   0    0 2448 2048]
 [   0    0 2448 2048]
 [   0    0 2448 2048]
 [   0    0 2448 2048]]


## Create a multiview tree
Here is to set the `connectivity` between cameras. Give the hint which pairs of cameras are likely share/(connectivity) most views field of ball. In general case, all cameras share most common view field. You can use default config. 

For example, `0` share with `2`, `0` not with `1`, `1` share with `2`. You can set `"minimal_tree":{[0, 2], [1, 2]}` to global connection.

Note that this `connectivity` define only works when calibration, and won't affect next step 2D<->3D transfrom.


In [3]:
nview=len(matdata['views_xywh'])
minimal_tree = [[i,i+1] for i in range(nview-1)]
setup_dict = {"views": list(range(nview)),
              "minimal_tree": minimal_tree}
setup_json = 'setup.json'
json.dump(setup_dict, open(setup_json, 'w'), indent=4)
print(setup_dict)

{'views': [0, 1, 2, 3], 'minimal_tree': [[0, 1], [1, 2], [2, 3]]}


## Downsample the ball sample to <=1000
It will do
- Downsample ball sample to 1000
- Pack all nessary informtion into one file

In [4]:
from matpkl2ballpkl import convert as convert_matpkl2ballpkl
# matfile = '/home/gxj/Desktop/gxj/calibration_playground/ball_label/ball.matpkl'
# intrinsics_json = '/home/gxj/Desktop/gxj/calibration_playground/intrinsic_calib_frames/intrinsics_calib.json'
setup_json = 'setup.json'
convert_matpkl2ballpkl(matfile, intrinsics_json, setup_json)

(4, 100, 2)
ind_3notnan: [ True  True  True  True]
Video file /home/gxj/Desktop/gxj/calibration_playground/ball_label/ball.mp4 does not exist
python -m lilab.multiview_scripts_dev.s3_ballpkl2calibpkl /home/gxj/Desktop/gxj/calibration_playground/ball_label/ball.ballpkl


'/home/gxj/Desktop/gxj/calibration_playground/ball_label/ball.ballpkl'

## Content of `ballpkl`
- **setup**: copy from `setup.json`
- **intrinsics**: copy from `intrinsic_calib.json`
- **landmarks_move_xy**: downsample ball xy position, (NVIEW, 1000, XY)
- **background_img**: (NVIEW, )x(H,W,3) numpy array for background image each view, default not set
- *landmarks_global_xy*, *landmarks_global_cm*, *global_iframe*: obsoleted


In [8]:
ballpklfile = matfile.replace('.matpkl', '.ballpkl')
ballpkl = pickle.load(open(ballpklfile, 'rb'))
print(ballpkl.keys())
print(ballpkl['landmarks_move_xy'].shape)
print(ballpkl['intrinsics'])


dict_keys(['landmarks_global_xy', 'landmarks_move_xy', 'global_iframe', 'landmarks_global_cm', 'background_img', 'setup', 'intrinsics'])
(4, 100, 2)
{'0': {'date': '2023-12-20 13:36:00', 'description': 'by opencv', 'K': [[1882, 0, 1231], [0, 1878, 1088], [0, 0, 1]], 'dist': [-0.182, 0.114, 0.0, 0.0, 0.0], 'image_shape': [2048, 2448]}, '1': {'date': '2023-12-20 13:36:00', 'description': 'by opencv', 'K': [[1946, 0, 1207], [0, 1944, 1067], [0, 0, 1]], 'dist': [-0.11, 0.372, 0.0, 0.0, 0.0], 'image_shape': [2048, 2448]}, '2': {'date': '2023-12-20 13:36:00', 'description': 'by opencv', 'K': [[1874, 0, 1207], [0, 1862, 1137], [0, 0, 1]], 'dist': [-0.18, 0.231, 0.0, 0.0, 0.0], 'image_shape': [2048, 2448]}, '3': {'date': '2023-12-20 13:36:00', 'description': 'by opencv', 'K': [[1843, 0, 1121], [0, 1820, 1163], [0, 0, 1]], 'dist': [-0.233, 0.206, 0.0, 0.0, 0.0], 'image_shape': [2048, 2448]}}


In [6]:
ballpklfile = '/home/gxj/Desktop/gxj/calibration_playground/ball_label/ball_1221.ballpkl'
ballpkl = pickle.load(open(ballpklfile, 'rb'))
print(ballpkl.keys())
print(ballpkl['landmarks_move_xy'].shape)
print(ballpkl['intrinsics'])

dict_keys(['landmarks_global_xy', 'landmarks_move_xy', 'global_iframe', 'landmarks_global_cm', 'background_img', 'setup', 'intrinsics'])
(4, 100, 2)
{'0': {'date': '2023-12-20 13:36:00', 'description': 'by opencv', 'K': [[1882, 0, 1231], [0, 1878, 1088], [0, 0, 1]], 'dist': [-0.182, 0.114, 0.0, 0.0, 0.0], 'image_shape': [2048, 2448]}, '1': {'date': '2023-12-20 13:36:00', 'description': 'by opencv', 'K': [[1946, 0, 1207], [0, 1944, 1067], [0, 0, 1]], 'dist': [-0.11, 0.372, 0.0, 0.0, 0.0], 'image_shape': [2048, 2448]}, '2': {'date': '2023-12-20 13:36:00', 'description': 'by opencv', 'K': [[1874, 0, 1207], [0, 1862, 1137], [0, 0, 1]], 'dist': [-0.18, 0.231, 0.0, 0.0, 0.0], 'image_shape': [2048, 2448]}, '3': {'date': '2023-12-20 13:36:00', 'description': 'by opencv', 'K': [[1843, 0, 1121], [0, 1820, 1163], [0, 0, 1]], 'dist': [-0.233, 0.206, 0.0, 0.0, 0.0], 'image_shape': [2048, 2448]}}


## !! The most critical setup
## Multiview extrinsic calibration.
The main precessing is :
1. Pair each two pair cameras in the `setup.json/mini_tree`.
2. Concat all views. Get a rough relative pose.
3. Remove outlier samples. Use bundle ajustment to refine relative poses.

![image.png](attachment:image.png)

The outcome is `*.calibpkl` file.

In [9]:
from ballpkl2calibpkl import main_calibrate
calibpklfile = main_calibrate(ballpklfile, skip_global=True, skip_camera_intrinsic=True)

2023-12-21 17:41:26,169 [root] -------------------------------------------------
2023-12-21 17:41:26,170 [root] Computing robust relative pose for pair 0->1
2023-12-21 17:41:26,171 [root] Initial relative pose:
2023-12-21 17:41:26,176 [root] Computing relative pose of pair [0, 1]:
2023-12-21 17:41:26,176 [root] 	0 out of 100 points considered outliers.
2023-12-21 17:41:26,179 [root] 	Residual error: 138.67379291716188
2023-12-21 17:41:26,181 [root] 	Sampson distance: 11468.449755164826
2023-12-21 17:41:26,182 [root] Number of additional paths found: 2
2023-12-21 17:41:26,186 [root] 	Computing relative pose of pair [0, 2]:
2023-12-21 17:41:26,186 [root] 		0 out of 100 points considered outliers.
2023-12-21 17:41:26,189 [root] 		Residual error: 190.19146472600684
2023-12-21 17:41:26,191 [root] 		Sampson distance: 8907.50532170125
2023-12-21 17:41:26,194 [root] 	Computing relative pose of pair [2, 1]:
2023-12-21 17:41:26,195 [root] 		0 out of 100 points considered outliers.
2023-12-21 17:

Fixing camera instrincis!


100%|██████████████████████████████████████| 100/100 [00:00<00:00, 22741.98it/s]
2023-12-21 17:41:27,394 [root] The preparation of the input data took: 0.01s
2023-12-21 17:41:27,394 [root] Sizes:
2023-12-21 17:41:27,396 [root] 	 camera_params: (4, 15)
2023-12-21 17:41:27,397 [root] 	 points_3d: (100, 3)
2023-12-21 17:41:27,397 [root] 	 points_2d: (400, 2)
2023-12-21 17:41:27,662 [root] Early Outlier rejection:
2023-12-21 17:41:27,664 [root] 	 threshold outliers: 100
2023-12-21 17:41:27,665 [root] 	 Number of points considered outliers: 399
2023-12-21 17:41:27,666 [root] !!!!!!!!!!!!!!!!!!!!
2023-12-21 17:41:27,667 [root] More than half of the data points have been considered outliers! Something may have gone wrong.
2023-12-21 17:41:27,668 [root] !!!!!!!!!!!!!!!!!!!!
2023-12-21 17:41:27,840 [multiview_calib.bundle_adjustment_scipy_short]    Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality
2023-12-21 17:41:27,842 [multiview_calib.bundle_adjustment_scip

RuntimeError: All 3D points have been discarded/considered outliers.

In [7]:
from ballpkl2calibpkl import main_calibrate
ballpklfile = '/home/gxj/Desktop/gxj/calibration_playground/ball_label/ball_1221.ballpkl'
calibpklfile = main_calibrate(ballpklfile, skip_global=True, skip_camera_intrinsic=True)

2023-12-21 17:40:27,820 [root] -------------------------------------------------
2023-12-21 17:40:27,822 [root] Computing robust relative pose for pair 0->1
2023-12-21 17:40:27,823 [root] Initial relative pose:
2023-12-21 17:40:27,865 [root] Computing relative pose of pair [0, 1]:
2023-12-21 17:40:27,866 [root] 	0 out of 100 points considered outliers.
2023-12-21 17:40:27,870 [root] 	Residual error: 138.67379291716188
2023-12-21 17:40:27,872 [root] 	Sampson distance: 11468.449755164826
2023-12-21 17:40:27,873 [root] Number of additional paths found: 2
2023-12-21 17:40:27,878 [root] 	Computing relative pose of pair [0, 2]:
2023-12-21 17:40:27,879 [root] 		0 out of 100 points considered outliers.
2023-12-21 17:40:27,883 [root] 		Residual error: 190.19146472600684
2023-12-21 17:40:27,885 [root] 		Sampson distance: 8907.50532170125
2023-12-21 17:40:27,888 [root] 	Computing relative pose of pair [2, 1]:
2023-12-21 17:40:27,889 [root] 		0 out of 100 points considered outliers.
2023-12-21 17:

Fixing camera instrincis!


100%|██████████████████████████████████████| 100/100 [00:00<00:00, 31165.88it/s]
2023-12-21 17:40:29,202 [root] The preparation of the input data took: 0.02s
2023-12-21 17:40:29,203 [root] Sizes:
2023-12-21 17:40:29,203 [root] 	 camera_params: (4, 15)
2023-12-21 17:40:29,204 [root] 	 points_3d: (100, 3)
2023-12-21 17:40:29,204 [root] 	 points_2d: (400, 2)
2023-12-21 17:40:29,395 [root] Early Outlier rejection:
2023-12-21 17:40:29,396 [root] 	 threshold outliers: 100
2023-12-21 17:40:29,397 [root] 	 Number of points considered outliers: 399
2023-12-21 17:40:29,398 [root] !!!!!!!!!!!!!!!!!!!!
2023-12-21 17:40:29,399 [root] More than half of the data points have been considered outliers! Something may have gone wrong.
2023-12-21 17:40:29,399 [root] !!!!!!!!!!!!!!!!!!!!
2023-12-21 17:40:29,575 [multiview_calib.bundle_adjustment_scipy_short]    Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality
2023-12-21 17:40:29,576 [multiview_calib.bundle_adjustment_scip

RuntimeError: All 3D points have been discarded/considered outliers.

## Content of the outcome `calibpkl` file
- **ba_poses**: all intrinsic & extrinsic parameters. `K` (3, 3), `dist`: (K1_K2_T1_T2_K3,), `t` (3,), `R` (3, 3)

In [ ]:
calibpkl = pickle.load(open(calibpklfile, 'rb'))
print(calibpkl.keys())

print('ba_poses', calibpkl['ba_poses'])
calibpkl['ba_poses'][0]

## Next step **[3] world axes registration** and **[4] 2D-3D mutual transformation**.
The **world axes registration** is optional. If you don't want to do it, just skip it.